### Análisis de Horas Críticas del Mercado para el Balance de Potencia ###
**Author:** Aminadab Cordova

**Date:** 2024-05-02

In [ ]:
# Módulos requeridos
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Conexión servidor

In [ ]:
# Consulta SQL para las horas criticas del año de producción
sql_query = """SELECT * FROM public."Horas_Criticas_AP" """

# Ejecutar la consulta y cargar los resultados en un DataFrame
df_hc_ap = pd.read_sql(sql_query, cnx_mbp)